In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e10/sample_submission.csv
/kaggle/input/playground-series-s5e10/train.csv
/kaggle/input/playground-series-s5e10/test.csv
/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_10k.csv
/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_2k.csv
/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_100k.csv


In [ ]:
!uv pip install -U autogluon > /dev/null

Using Python 3.11.13 environment at: /usr
Resolved 222 packages in 4.96s                          
⠙ Preparing packages... (0/144)                                                 
⠙ Preparing packages... (0/144)--     0 B/6.01 MiB                      
⠙ Preparing packages... (0/144)-- 14.88 KiB/6.01 MiB                    
tokenizers ------------------------------     0 B/2.97 MiB
⠙ Preparing packages... (0/144)-- 14.88 KiB/6.01 MiB                    
tokenizers ------------------------------ 14.91 KiB/2.97 MiB
⠙ Preparing packages... (0/144)-- 14.88 KiB/6.01 MiB                    
pydantic-core ------------------------------     0 B/1.98 MiB
tokenizers ------------------------------ 14.91 KiB/2.97 MiB
⠙ Preparing packages... (0/144)-- 14.88 KiB/6.01 MiB                    
pydantic-core ------------------------------ 14.90 KiB/1.98 MiB
tokenizers ------------------------------ 14.91 KiB/2.97 MiB
⠙ Preparing packages... (0/144)-- 14.88 KiB/6.01 MiB                    
aiohttp    ---

In [15]:
from autogluon.tabular import TabularDataset, TabularPredictor
from pathlib import Path

In [16]:
path = Path('/kaggle/input/playground-series-s5e10/')
path

PosixPath('/kaggle/input/playground-series-s5e10')

In [17]:
train_df = pd.read_csv(path/'train.csv',index_col='id')
test_df = pd.read_csv(path/'test.csv',index_col='id')
sub_df = pd.read_csv(path/'sample_submission.csv')
og_df_a = pd.read_csv('/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_100k.csv')

In [18]:
og_df_a

,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,rural,2,0.29,70,night,rainy,False,True,evening,False,False,1,0.64
1,highway,1,0.34,25,dim,clear,False,False,morning,False,False,3,0.27
2,rural,2,0.76,70,night,foggy,True,False,evening,True,True,1,0.76
3,rural,3,0.37,70,night,foggy,True,False,morning,False,True,0,0.60
4,highway,3,0.39,45,dim,rainy,False,True,morning,False,False,0,0.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,rural,2,0.61,60,dim,foggy,False,True,evening,False,False,1,0.54
99996,rural,2,0.52,35,dim,foggy,True,True,afternoon,False,False,1,0.18
99997,rural,2,0.08,70,daylight,clear,False,False,evening,True,False,1,0.20
99998,rural,1,0.83,70,dim,foggy,False,True,morning,False,True,0,0.58


In [ ]:
train_df

In [19]:
TabularPredictor.fit??

Signature:
TabularPredictor.fit(
    self,
    train_data: 'pd.DataFrame | str',
    tuning_data: 'pd.DataFrame | str' = None,
    time_limit: 'float' = None,
    presets: 'list[str] | str' = None,
    hyperparameters: 'dict | str' = None,
    feature_metadata: 'str | FeatureMetadata' = 'infer',
    infer_limit: 'float' = None,
    infer_limit_batch_size: 'int' = None,
    fit_weighted_ensemble: 'bool' = True,
    fit_full_last_level_weighted_ensemble: 'bool' = True,
    full_weighted_ensemble_additionally: 'bool' = False,
    dynamic_stacking: 'bool | str' = False,
    calibrate_decision_threshold: 'bool | str' = 'auto',
    num_cpus: 'int | str' = 'auto',
    num_gpus: 'int | str' = 'auto',
    fit_strategy: "Literal['sequential', 'parallel']" = 'sequential',
    memory_limit: 'float | str' = 'auto',
    callbacks: 'list[AbstractCallback]' = None,
    **kwargs,
) -> "'TabularPredictor'"
Source:   
    @apply_presets(tabular_presets_dict, tabular_presets_alias)
    def fit(
        se

In [20]:
target = 'accident_risk'
eval_metric = 'rmse'
#train_data = train
train_data = train_df
#Time_limit = 3600*11
Time_limit = 900
problem_type='regression'

In [ ]:
predictor = TabularPredictor(
    label=target, 
    eval_metric=eval_metric,
    verbosity=1,
    problem_type=problem_type
).fit(
    train_data, 
    presets='best_quality',
    excluded_model_types=['KNN'], 
    time_limit=Time_limit,
    dynamic_stacking=False,  # ← ADD THIS
    ag_args_fit={'num_gpus': 1}
)

No path specified. Models will be saved in: "AutogluonModels/ag-20251028_140523"
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.Tensor size changed, may indicate binary incompatibility. Expected 64 from C header, got 80 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.ChunkedArray size changed, may indicate binary incompatibility. Expected 64 from C header, got 72 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib._Tabular size changed, may indicate binary incompatibility. Expected 24 from C header, got 32 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.Table size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.NativeFile size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib

In [ ]:
%%time
predictor.leaderboard()

In [ ]:
%%time
autogluon_preds = predictor.predict(test_df)

In [ ]:
autogluon_preds

In [ ]:
!rm submission.csv
sub_df['Price'] = autogluon_preds.values
sub_df.to_csv('submission.csv', index=False)
sub_df.head()